In [60]:
import pandas as pd
import warnings


In [61]:
def wrangle(file):

    #read csv
    df = pd.read_csv(file)

    #subset data with class
    mask_a = df['Class'].str.contains('Development')

    #subset data with spending only at central level
    mask_b = df['Adm1'].str.contains('Central')

    df = df[mask_a & mask_b]

    #return dataframe
    return df

In [62]:
df = wrangle('kenya-boost-2006-2018.csv')
df.head()

/tmp/ipykernel_4199/2796509837.py:4: DtypeWarning: Columns (0,1,2,3,4,5,6,7,9,10,11,12,15,16,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


,Year,Class,Adm1,Adm2,Adm3,Adm4,Adm5,Adm6,Geo1,Econ0,econ1,econ2,econ3,econ4,FndSrce1,function1,function2,Estimates,Revised,Executed
0,2005/6,Development,Central,110 Ministry of Agriculture,110.103 Facilitation And Supply Of Agriculture...,10.103.761 National Crops and Horticultural Re...,NaN,NaN,NaN,Expenditures,Other recurrent,Goods and Services,2211200 Fuel Oil and Lubricants,NaN,00 Domestic RESOURCES,Agriculture and Rural Development,Agriculture and Rural Development,"18,318,600","14,062,700","5,893,860"
1,2005/6,Development,Central,110 Ministry of Agriculture,110.103 Facilitation And Supply Of Agriculture...,10.103.761 National Crops and Horticultural Re...,NaN,NaN,NaN,Expenditures,Other recurrent,Goods and Services,2211300 Other o Perating Expenses,NaN,00 Domestic RESOURCES,Agriculture and Rural Development,Agriculture and Rural Development,"12,799,450","9,599,587","3,312,808"
2,2005/6,Development,Central,110 Ministry of Agriculture,110.103 Facilitation And Supply Of Agriculture...,10.103.761 National Crops and Horticultural Re...,NaN,NaN,NaN,Expenditures,Other recurrent,Goods and Services,2220200 Routine Maintenance - Other Assets,NaN,00 Domestic RESOURCES,Agriculture and Rural Development,Agriculture and Rural Development,"7,218,400","5,652,150","2,911,400"
3,2005/6,Development,Central,110 Ministry of Agriculture,110.103 Facilitation And Supply Of Agriculture...,10.103.761 National Crops and Horticultural Re...,NaN,NaN,NaN,Expenditures,Other recurrent,Goods and Services,2220200 Routine Maintenance - Other Assets,NaN,00 Domestic RESOURCES,Agriculture and Rural Development,Agriculture and Rural Development,"10,025,827","8,126,744","4,501,504"
4,2005/6,Development,Central,110 Ministry of Agriculture,110.103 Facilitation And Supply Of Agriculture...,10.103.761 National Crops and Horticultural Re...,NaN,NaN,NaN,Expenditures,Capital,Capital,3110200 Construction of Buidings,NaN,00 Domestic RESOURCES,Agriculture and Rural Development,Agriculture and Rural Development,"500,000","1,130,625","95,000"


In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19226 entries, 0 to 263426
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Year         19226 non-null  object 
 1   Class        19226 non-null  object 
 2   Adm1         19226 non-null  object 
 3   Adm2         19226 non-null  object 
 4   Adm3         18975 non-null  object 
 5   Adm4         19226 non-null  object 
 6   Adm5         14178 non-null  object 
 7   Adm6         1994 non-null   object 
 8   Geo1         0 non-null      float64
 9   Econ0        19226 non-null  object 
 10  econ1        19226 non-null  object 
 11  econ2        19226 non-null  object 
 12  econ3        19226 non-null  object 
 13  econ4        0 non-null      float64
 14  FndSrce1     19226 non-null  object 
 15  function1    19226 non-null  object 
 16  function2    19226 non-null  object 
 17   Estimates   19226 non-null  object 
 18   Revised     19226 non-null  object 
 19   Execute